<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Emilio Roa


### **Link de repositorio de GitHub:** `https://github.com/ERoa98/MDS7202-Emilio-Roa`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [ ]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 1.1 MB 38.0 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=a0cc473901addb0c54bf5ed3814bed91017ca3cbc7b86d635cea15f6ed2b0ab8
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=8ae18ba7f0e6da03f791ad5d09284d40ff33298fb67df7deec6cf92c0f7fc620
  Stored in directory: /root/.cache/pip/wheels/19/bc/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [ ]:
df_comics = pd.read_csv('df_comics.csv')
df_comics_no_label = pd.read_csv('comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [ ]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [ ]:
#### Código aquí ####
from sklearn.compose import ColumnTransformer


atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

bow = CountVectorizer(tokenizer= StemmerTokenizer(),
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )


ct = ColumnTransformer(
    transformers=[   
        ('CountVec', bow, 'history_text'),
        ('MinMaxScaler', MinMaxScaler(), atributos_de_interes)
        ])




In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(df_comics['history_text'])

In [ ]:
ct.fit(df_comics)

ColumnTransformer(transformers=[('CountVec',
                                 CountVectorizer(ngram_range=(1, 2),
                                                 tokenizer=<__main__.StemmerTokenizer object at 0x7f1f6d752490>),
                                 'history_text'),
                                ('MinMaxScaler', MinMaxScaler(),
                                 ['intelligence_score', 'strength_score',
                                  'speed_score', 'durability_score',
                                  'power_score', 'combat_score'])])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [ ] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [ ] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [ ]:
#### Código aquí ####

baseline_pipe = Pipeline(
    [
    ("preprocesamiento", ct),
    ("Feature_Selector", SelectPercentile(f_classif, percentile=90)),
    ("clf", MultinomialNB())
    ]
)

#df_comics['alignment'].replace({'Good': 1, 'Bad': 0, 'Neutral': 2}, inplace= True)

X_train, X_test, y_train, y_test = train_test_split(df_comics.drop(columns=['alignment', 'Unnamed: 0']),
                                                    df_comics['alignment'],
                                                    test_size=0.33,
                                                    shuffle = True,
                                                    stratify = df_comics['alignment'],
                                                    random_state=42)



In [ ]:
y_train

424        Good
972         Bad
991        Good
47          Bad
1062    Neutral
         ...   
267         Bad
846        Good
419         Bad
1241       Good
477         Bad
Name: alignment, Length: 860, dtype: object

In [ ]:
baseline_pipe = baseline_pipe.fit(X_train, y_train)
y_pred = baseline_pipe.predict(X_test)

from sklearn.metrics import confusion_matrix
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[ 19 123   0]
 [  3 243   0]
 [  3  34   0]] 

              precision    recall  f1-score   support

         Bad       0.76      0.13      0.23       142
        Good       0.61      0.99      0.75       246
     Neutral       0.00      0.00      0.00        37

    accuracy                           0.62       425
   macro avg       0.46      0.37      0.33       425
weighted avg       0.61      0.62      0.51       425



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Respuesta: Dentro de este primer modelo clasificador se puede comentar que a primera vista los resultados son bastantes malos sobre todo en la clasificación de las clases minoritarias, esto debido a que el modelo se sobreajustó en la clase "Good" que es la que presenta el mejor recall. El problema con las clases minoritarias se refleja principalmente en los bajos recall, lo cual significa que no está identificando los casos reales de esa clase (sobre todo en Bad donde la precision es 0.76 pero con un bajísimo recall)

¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?

Respuesta: No es recomendable utilizar este clasificador para predecir si BatCow es Bueno, Malo o Neutral ya que el modelo se encuentra sobre entrenado para la clase Bueno.

In [ ]:

from sklearn.dummy import DummyClassifier
dummy_pipe = Pipeline(
    [
    ("preprocesamiento", ct),
    ("Feature_Selector", SelectPercentile(f_classif, percentile=90)),
    ("Dummy", DummyClassifier(strategy="stratified"))
    ]
)
dummy_pipe = dummy_pipe.fit(X_train, y_train)
y_pred = dummy_pipe.predict(X_test)

from sklearn.metrics import confusion_matrix
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[ 58  68  16]
 [ 79 142  25]
 [ 12  21   4]] 

              precision    recall  f1-score   support

         Bad       0.39      0.41      0.40       142
        Good       0.61      0.58      0.60       246
     Neutral       0.09      0.11      0.10        37

    accuracy                           0.48       425
   macro avg       0.36      0.36      0.36       425
weighted avg       0.49      0.48      0.49       425





Respuesta: El clasificador Dummy, configurado con la estrategia de tipo "Stratified", no toma en consideración los atributos de los datos sino que construye distribuciones multinomiales parametrizadas a partir de las probabilidades prior de las clases. Esto es jugar mucho con las probabilidad y dado que no son muchos ejemplos esto no se prodría considera representativo y así también hace referencia su nombre "dummy".

Ahora, contrastando este clasificador dummy con el baseline obtenido en la celda anterior, se tiene que los resultados no se encuentran muy alejados y tomando en consideración que el dummy es el clasificador más básico a realizar el baseline se encuentra en una escala de resultados sumamente pobre.

¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?
Respuesta: En base a lo comentado anteriormente, el clasificador entrenado no mejora los resultados del dummy

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

In [ ]:
pipe = Pipeline(
    [
    ("preprocesamiento", ct),
    ("Feature_Selector", SelectPercentile(f_classif, percentile=90)),
    ("classifier", RandomForestClassifier())
    ]
)

pipe.get_params()

{'memory': None,
 'steps': [('preprocesamiento', ColumnTransformer(transformers=[('CountVec',
                                    CountVectorizer(ngram_range=(1, 2),
                                                    tokenizer=<__main__.StemmerTokenizer object at 0x7f1f6d752490>),
                                    'history_text'),
                                   ('MinMaxScaler', MinMaxScaler(),
                                    ['intelligence_score', 'strength_score',
                                     'speed_score', 'durability_score',
                                     'power_score', 'combat_score'])])),
  ('Feature_Selector', SelectPercentile(percentile=90)),
  ('classifier', RandomForestClassifier())],
 'verbose': False,
 'preprocesamiento': ColumnTransformer(transformers=[('CountVec',
                                  CountVectorizer(ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f1f6d752490>),
  

**Respuesta:**

In [ ]:
#### Código aquí ####
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

params = [
       # clasificador 1 + hiperparámetros
       {'classifier' : [RandomForestClassifier()],
        'classifier__n_estimators': [2, 5, 10],
        'preprocesamiento__CountVec__ngram_range': [(1,1), (1, 2), (1,3)],
        'preprocesamiento__CountVec__max_df': [0.8, 0.9, 1.0],
        'preprocesamiento__CountVec__min_df': [0.0, 0.1, 0.2],
        'Feature_Selector__percentile': [20, 40, 60, 80]

       },
       # clasificador 1 + hiperparámetros    
       {'classifier': [MultinomialNB()],
        'preprocesamiento__CountVec__ngram_range': [(1,1), (1, 2), (1,3)],
        'preprocesamiento__CountVec__max_df': [0.8, 0.9, 1.0],
        'preprocesamiento__CountVec__min_df': [0.0, 0.1, 0.2],
        'Feature_Selector__percentile': [20, 40, 60, 80]
       },
  
       # clasificador 1 + hiperparámetros
       {'classifier': [GradientBoostingClassifier(random_state=666)],
        'preprocesamiento__CountVec__ngram_range': [(1,1), (1, 2), (1,3)],
        'preprocesamiento__CountVec__max_df': [0.8, 0.9, 1.0],
        'preprocesamiento__CountVec__min_df': [0.0, 0.1, 0.2],
        'Feature_Selector__percentile': [20, 40, 60, 80]
       }
       ]
        
search = HalvingGridSearchCV(pipe, params,
                             verbose=10,
                             cv=3,
                             n_jobs=-1, scoring="balanced_accuracy",
                             random_state=0).fit(X_train, y_train)
search.best_params_  

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 18
max_resources_: 860
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 540
n_resources: 18
Fitting 3 folds for each of 540 candidates, totalling 1620 fits
----------
iter: 1
n_candidates: 180
n_resources: 54
Fitting 3 folds for each of 180 candidates, totalling 540 fits
----------
iter: 2
n_candidates: 60
n_resources: 162
Fitting 3 folds for each of 60 candidates, totalling 180 fits
----------
iter: 3
n_candidates: 20
n_resources: 486
Fitting 3 folds for each of 20 candidates, totalling 60 fits


{'Feature_Selector__percentile': 60,
 'classifier': GradientBoostingClassifier(random_state=666),
 'preprocesamiento__CountVec__max_df': 0.8,
 'preprocesamiento__CountVec__min_df': 0.2,
 'preprocesamiento__CountVec__ngram_range': (1, 3)}

In [ ]:
#Modelo final

new_pipe = search.best_estimator_

new_pipe = new_pipe.fit(X_train, y_train)
y_pred = new_pipe.predict(X_test)

from sklearn.metrics import confusion_matrix
print("Matriz de confusión\n\n", confusion_matrix(y_test, y_pred), "\n")
print(classification_report(y_test, y_pred))

Matriz de confusión

 [[ 57  83   2]
 [ 38 205   3]
 [ 17  18   2]] 

              precision    recall  f1-score   support

         Bad       0.51      0.40      0.45       142
        Good       0.67      0.83      0.74       246
     Neutral       0.29      0.05      0.09        37

    accuracy                           0.62       425
   macro avg       0.49      0.43      0.43       425
weighted avg       0.58      0.62      0.59       425



La mejor combinación encontrada para el modelo según la grilla resultó ser el clasificador GradientBoosting(). En cuanto a las features utilizadas sólo se utilizó las que abarcaban el 60%, por lo que un 40% de las columnas no son relevantes a este problema. Por otra parte, el modelo arroja utilizar la mayor cantidad de n-gramas, lo cual tiene sentido dado que es importante tener el mayor contexto posible al vectorizar. Por último, en los mejores parámetros del CountVectorizer se obtuvo mejores resultados con las combinaciones de min_df=0.2 y max_df=0.8, lo cual significa que palabras muy repetidas no son consideradas (sobre el 80%) y pocas repetidas tampoco (bajo el 20%).

Comparandolo con el pipeline baseline, obtenemos un accuracy identico de 62%. Sin embargo, en este nuevo pipeline se obtienen mejores resultados globales para las 3 clases y el modelo no está tan dominado por la clase 'Good'. Por lo menos, pudo clasificar 2 heroes como neutros, aunque evidentemente los resultados no son tan buenos y para resolver de mejor forma esta tarea posiblemente sea necesario implementar modelos con mayor capacidad de procesamiento de secuencias como redes neuronales recurrentes.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
df = df_comics_no_label.dropna(subset = ['history_text'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop_duplicates()

df.head()

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,514A (Gotham),Bruce Wayne,NaN,10,"He was one of the many prisoners of Indian Hill to be transferred to another facility upstate on the orders of The Court. However, Fish Mooney hijacks the bus and drives it into Gotham City, where the bus crashes when fired upon by Butch Gilzean and his gang. After the mobsters flee at the sight of a resurrected Fish, she herself leaves the scene. An elderly hobo lady hears the cries of the prisoners and releases them, only to be horrified by their monstrous appearance. The monsters depart the prison van and make their way into Gotham City. The Bruce Wayne look-alike departs the bus and thanks the elderly woman before entering the city himself The look-a-like later saw Selina Kyle and Ivy Pepper give some money to a street gang. After Selina Kyle leaves to go meet up with Fish Mooney and her gang, Ivy is visited by the Bruce Wayne look-a-like. He frantically asks her who Bruce Wayne is, confused as to why he looks almost identical to him. Confused and terrified, Ivy runs off to inform Selina of his odd behavior. The look-a-like later snuck into Wayne Manor where Bruce and Alfred Pennyworth find him and are surprised by his appearance. He begs them not to hurt him.While helping himself to some food, the look-a-like stated to Bruce that he was known as Subject 514A and states that he never knew about the other Indian Hill subjects until he was loaded onto the bus that he was later freed from. Bruce objected to Alfred's suggestion to turn him over to someone. Subject 514A later visits Bruce's sparing match with Alfred. Taking his shot with Alfred, Subject 514A managed to easily counter Alfred's moves. When Subject 514A managed to get punched in the nose, Alfred saw that he didn't feel pain. Alfred and Bruce then noticed the scar on the back of his neck. Bruce looks for a clue for the scar on Subject 514A and hides him when Selina Kyle came into view. After watching Bruce and Selina's conversation, Subject 514A started to begin mimicking Bruce's voice. Later that night while Bruce is sleeping, Subject 514A starts cutting his hair. The next morning, Bruce and Alfred find out what Subject 514A did upon finding his cut hair and Bruce's missing clothes. Bruce tells Alfred that they need to find him. Subject 514A arrives in one of the cars from Wayne Manor and invites Selina Kyle to have food with him. While Selina is suspicious on why ""Bruce"" is acting strange, she goes with him anyway.The next day, Selina and Subject 514A walked down the streets as they approach a bar. As Selina goes into the bar, Subject 514A is told to go around back. As Selina is about to have her fingers cut off by the two owners, Subject 514A comes in and beats up the two men. Selina is then cut loose by Subject 514A. After getting away, Selina discovers that Subject 514A is wounded. Upon noticing the scars on him, Selina sees that he is not Bruce. Selina stitches up Subject 514A who states that he has no knowledge of why he was made to look like him and claims that Bruce might have an interest in Selina after watching them from the halls. After being told by Selina about what happened to Firefly and Ivy, Subject 514A then kisses Selina. Bruce and Alfred catch up to Subject 514A who claims that Bruce doesn't know what he wants. Subject 514A claims that he doesn't know why he was created and states that he doesn't belong in Gotham City. He then jumps off the building telling them to say goodbye to Selina for him. Bruce and Alfred then see Subject 514A walk off into the streets. Subject 514A is later visited by Kathryn and Talon while walking on the streets. Kathryn states to Subject 514A that they are his ""real parents"" who have been looki

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
#Vergil, Gorilla Girl y Batcow
Vergil = df.loc[df['name'] == 'Vergil']
Gorilla_Girl = df.loc[df['name'] == 'Gorilla Girl']
Batcow = df.loc[df['name'] == 'Batcow']
print(Batcow['history_text'])
print(Gorilla_Girl['history_text'])
print(Vergil['history_text'])

16    Bat-Cow was originally a cow that was found by Robin during an investigation on a slaughterhouse, which the cow was sent to. Robin wanted to keep the cow and named it Bat-Cow. Batman decided they would keep the cow. During a later investigation, Batman discovered it had mind-controlling radiation inside of it, leading him to attempt to get rid of the cow. Robin, however would not allow him to hurt the cow. After the events of Robin's death, Bat-Cow and Robin's dog felt a great loss in morale. Alfred Pennyworth and Nightwing had to take care of the cow. Later after Nightwing's disappearance, Alfred was the only caretaker of Bat-Cow. Bat-Cow, complete with cape, spent time on a Wayne Enterprises-owned dairy farm. It was there the cow encountered the cosmic-powered "Forever People." One of them seemed to gain guidance from Bat-Cow's actions, namely when Bat-Cow gave one of the Forever People a scarecrow's hat. Bat-Cow formed a friendship with Titus, a dog and household cat.
Name: hi

In [ ]:
Vergil_pred = new_pipe.predict(Vergil)
Gorilla_Girl_pred = new_pipe.predict(Gorilla_Girl)
Batcow_pred = new_pipe.predict(Batcow)

In [ ]:
print(Vergil_pred)
print(Gorilla_Girl_pred)
print(Batcow_pred)

['Good']
['Good']
['Bad']


Al realizar las predicciones, se puede ver que se predijo correctamente solo un personaje (Gorilla_Girl), ya que Batcow según se descripción debería ser conciderado como bueno y el modelo lo predice como malo, mientras que con Vergil es al revés.

Esto demuestra que el modelo pese a tener los mejores parámetros de las combinaciones que se probaron, sigue siendo deficiente para predecir correctamente cada personaje.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>